<a href="https://colab.research.google.com/github/XGHouFTW/py-cryptopredict/blob/main/StockPriceGaps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import datetime
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
transMonth = {"Jan":"01","Feb":"02","Mar":"03","Apr":"04","May":"05","Jun":"06","Jul":"07","Aug":"08","Sep":"09","Oct":"10","Nov":"11","Dec":"12"}

start = "2020-01-31" #from historical prices
end = "2022-01-31" #732 days
def trans(strdate):
    return datetime.datetime(int(strdate[0:4]), int(strdate[5:7]), int(strdate[8:10]))

In [3]:
def rowinsert(row_number, df, row_value):
    df1 = df[0:row_number]
    df2 = df[row_number:]
    df1.loc[row_number]=row_value
    df_result = pd.concat([df1, df2])
    df_result.index = [*range(df_result.shape[0])]

    return df_result

In [4]:
cryptopricekey = ["BTC","ETH","DOGE"]
cryptobasepath = "/content/drive/MyDrive/ISEF/Datasets/Historical Prices/" # + cryptopricekey + "-USD.csv"
stockpricekey = ["AAPL","AMC","DJI","GME","NASDAQ","S&P","TSLA"]
stockbasepath = "/content/drive/MyDrive/ISEF/Datasets/Stonks/Raw/" # + stockpricekey + "_historical.csv"

searchkey = ["bitcoin","ethereum","dogecoin","cryptocurrency","economy","finance","politics","pandemic"]
dfPrice = pd.DataFrame()
dfAddDate = pd.read_csv("/content/drive/MyDrive/ISEF/Datasets/Historical Prices/BTC-USD.csv")
dfPrice["Date"] = dfAddDate["Date"].apply(lambda text: trans(text))

In [5]:
for word in cryptopricekey:
  dfAdd = pd.read_csv(cryptobasepath + word + "-USD.csv")
  del dfAdd["Date"]
  dfAdd = dfAdd.rename(columns=lambda x: word + "-" + x)
  dfPrice = dfPrice.join(dfAdd)

In [6]:
for word in stockpricekey:
  dfAdd = pd.read_csv(stockbasepath + word + "_historical.csv")
  for i in range(len(dfAdd)):
    dfAdd.loc[i,"Date"] = trans(dfAdd.loc[i,"Date"])
  i = 0
  while (i < len(dfAdd)-1):
    diff = (dfAdd.loc[i+1,"Date"]-dfAdd.loc[i,"Date"]).days
    diff -= 1
    for j in range(diff):
      copied = dfAdd.iloc[i,:]
      copied["Date"] = copied["Date"] + datetime.timedelta(days = j+1)
      dfAdd = rowinsert(i+j+1, dfAdd, copied)
    i += 1 + diff  
  del dfAdd["Date"]
  del dfAdd["Unnamed: 0"]
  dfAdd = dfAdd.rename(columns=lambda x: word + "-" + x)
  dfPrice = dfPrice.join(dfAdd)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [10]:
dfPrice.to_csv("/content/drive/MyDrive/ISEF/Datasets/Stonks/Preprocessed/CryptoStockPrice.csv")